# Process Reflection Coefficient Data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
#from asd21libraries import seismic_functions as seis
import xarray as xr
import sys
import os
import binascii
import struct
from scipy import signal
from scipy.io import loadmat
import scipy.stats as stats
import time
from multiprocessing import Process

%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from scipy.sparse import spdiags
from scipy.sparse.linalg import lsqr as splsqr
from spgl1.lsqr import lsqr
from spgl1 import spgl1, spg_lasso, spg_bp, spg_bpdn, spg_mmv
from spgl1.spgl1 import norm_l1nn_primal, norm_l1nn_dual, norm_l1nn_project
from spgl1.spgl1 import norm_l12nn_primal, norm_l12nn_dual, norm_l12nn_project

P190file = 'MGL2104_PD12_CSs/MGL2104PD12.0.p190'
c = 1480
Fs = 500


## Inputs and Functions

In [ ]:
AirgunVols = [360,360,40,180,90,120,60,220,220,
                  360,360,40,180,90,120,60,220,220,
                  360,360,40,180,90,120,60,220,220,
                  360,360,40,180,90,120,60,220,220]
xg = [-8, -8, -4.77, -2.41, 0.23, 2.69, 5, 8, 8, 
      -8, -8, -4.77, -2.41, 0.23, 2.69, 5, 8, 8, 
      -8, -8, -4.77, -2.41, 0.23, 2.69, 5, 8, 8, 
      -8, -8, -4.77, -2.41, 0.23, 2.69, 5, 8, 8]
yg = [12, 12, 12, 12, 12, 12, 12, 12, 12, 
      4, 4, 4, 4, 4, 4, 4, 4, 4, 
      -4, -4, -4, -4, -4, -4, -4, -4, -4, 
      -12, -12, -12, -12, -12, -12, -12, -12, -12]

srcD = 15

if srcD == 9:
    zg = [9.55, 8.8, 8.8, 9, 9, 9, 9, 9.55, 8.8, 
          9.55, 8.8, 8.8, 9, 9, 9, 9, 9.55, 8.8, 
          9.55, 8.8, 8.8, 9, 9, 9, 9, 9.55, 8.8, 
          9.55, 8.8, 8.8, 9, 9, 9, 9, 9.55, 8.8]
elif srcD == 12:
    zg = [12.55, 11.8, 11.8, 12, 12, 12, 12, 12.55, 11.8, 
          12.55, 11.8, 11.8, 12, 12, 12, 12, 12.55, 11.8, 
          12.55, 11.8, 11.8, 12, 12, 12, 12, 12.55, 11.8, 
          12.55, 11.8, 11.8, 12, 12, 12, 12, 12.55, 11.8]
elif srcD == 15:
    zg = [15.55, 14.8, 14.8, 15, 15, 15, 15, 15.55, 14.8, 
          15.55, 14.8, 14.8, 15, 15, 15, 15, 15.55, 14.8, 
          15.55, 14.8, 14.8, 15, 15, 15, 15, 15.55, 14.8, 
          15.55, 14.8, 14.8, 15, 15, 15, 15, 15.55, 14.8]

geometry = [xg,yg,zg]

    
def readP190_2104(filename):
    '''
    Read P190 Navigation file for MGL2104
    '''
    import re

    # filename = '/media/asd21/My Passport/MGL2104_PD12_CSs/MGL2104PD12.0.p190'

    RECEIVER_NUMBER = 1200

    class nav:
        numCables = 0
        shotStart = 0
        shotEnd = 0
        depth = []
        vesselX = []
        vesselY = []
        sourceX = []
        sourceY = []
        tailX = []
        tailY = []       
        receiverX = []
        receiverY = []
        receiverZ = []

    file = open(filename, 'r')
    lines = file.readlines()
    receiverX = []
    receiverY = []
    receiverZ = []
    for line in lines:
        if line.startswith('H0101GENERAL'):
            nav.numCables = int(re.sub(r'.*(\d)\sCABLE.*', r'\1 ', line))

        elif line.startswith('H2600Line'):
            nav.shotStart, nav.shotEnd = [int(i) for i in line.split() if i.isdigit()]

        elif line.startswith('VGL') or line.startswith('VMGL'):
            res = line.split('W')
            xyd = (re.sub(r'\.(\d)', r'.\1 ', res[1])).split()
            nav.vesselX.append(float(xyd[0]))
            nav.vesselY.append(float(xyd[1]))
            if len(xyd) >= 3:
                nav.depth.append(float(xyd[2]))
            else:
                nav.depth.append(0)

        elif line.startswith('SGL') or line.startswith('SMGL'):
            res = line.split('W')
            xyd = (re.sub(r'\.(\d)', r'.\1 ', res[1])).split()
            nav.sourceX.append(float(xyd[0]))
            nav.sourceY.append(float(xyd[1]))

        elif line.startswith('CGL') or line.startswith('CMGL'):
            res = line.split('W')
            xyd = (re.sub(r'\.(\d)', r'.\1 ', res[1])).split()
            nav.tailX.append(float(xyd[0]))
            nav.tailY.append(float(xyd[1]))

        elif line.startswith('R'):
          # Add the current receiver arrays to nav and clear them 
            if len(receiverX) == RECEIVER_NUMBER * 1: #nav.numCables:
                nav.receiverX.append(receiverX)
                nav.receiverY.append(receiverY)
                nav.receiverZ.append(receiverZ)
                receiverX = []
                receiverY = []
                receiverZ = []

            length = 26
            receivers = [line[i: i + length] for i in range(1, length * 3 + 1, length)]
            for rec in receivers:
                xyz = (re.sub(r'\.(\d)', r'.\1 ', rec[4:])).split()
                receiverX.append(float(xyz[0]))
                receiverY.append(float(xyz[1]))
                if len(xyz) == 3:
                    receiverZ.append(float(xyz[2]))
                else:
                    receiverZ.append(0)

    return nav


def SigExp(geometry, volumes, angle, rtmp, ghost=False):

    AirgunSigs = np.load('AirgunSigsMGL2104_New.npz')
    AirgunDict = {}
    for volume in [40,60,90,120,180,220,360]:
        AirgunDict['Airgun'+str(volume)] = AirgunSigs['Airgun'+str(volume)]

    c = 1500
    Fs = 500
    Tend = 1
    df = 1/Tend
    
    f = np.arange(0,Fs+df,df)
    tmpLen = len(f)
    if np.mod(tmpLen,2) == 0:
        f = np.append(np.arange(0,Fs/2,df),np.arange(-Fs/2,0,df))
    else:
        f = np.append(np.arange(0,Fs/2+df,df),np.arange(-Fs/2,0,df))

    p = np.zeros(len(f),dtype=complex)

    xg = geometry[0]
    yg = geometry[1]
    zg = geometry[2]

    tshift = 2*rtmp/c-0.15

    for ii in range(len(volumes)):

        tmp = np.copy(AirgunDict['Airgun'+str(volumes[ii])])
        TMP = np.fft.fft(np.append(tmp,np.zeros(len(f)-len(tmp))))*np.exp(-ci*2*np.pi*f*(xg[ii]/c))

        x = 2*rtmp*np.cos(angle/180*np.pi) - xg[ii]
        y = yg[ii]
        z = 2*rtmp*np.sin(angle/180*np.pi) - zg[ii]
        rnew = np.sqrt(x**2 + y**2 + z**2)
        p += TMP*np.exp(ci*2*np.pi*f*(rnew/c-tshift))

    if ghost == True:
        for ii in range(len(volumes)):

            tmp = np.copy(AirgunDict['Airgun'+str(volumes[ii])])
            TMP = np.fft.fft(np.append(tmp,np.zeros(len(f)-len(tmp))))*np.exp(-ci*2*np.pi*f*(xg[ii]/c))

            x = 2*rtmp*np.cos(angle/180*np.pi) - xg[ii]
            y = yg[ii]
            z = 2*rtmp*np.sin(angle/180*np.pi) + zg[ii]
            rnew = np.sqrt(x**2 + y**2 + z**2)
            p += -TMP*np.exp(ci*2*np.pi*f*(rnew/c-tshift))

    srcSig = np.fft.ifft(p)

    return srcSig

## Load Peak Data and Build Receiver Outputs

In [ ]:
# Good receivers
recGood = np.concatenate( ( np.arange(10,80,10), np.array([85]), np.arange(95,140,10), np.arange(145,190,10), np.arange(195,210,10), np.arange(215,591,10) ), axis=0)
recAll = np.arange(0,1200,5)

rr = 0

for recNum in recAll:
    print(recNum)
    
    recFile = 'MGL2104_SEL_Layers/MGL2104_RefCoeffs_Rec' + str(recNum) + '.nc'

    try:
        ds_disk = xr.open_dataset(recFile)
    except:
        continue
    daH = ds_disk["Heights"]
    daT = ds_disk["Times"]
    shots = daH["shot"]
    pkNum = daH["peak number"]
    freqs = daH["frequency"][0:1]
    
    t = np.arange(0,15*Fs+1,1)/Fs
    
    if rr == 0:
        Rdata = np.zeros([len(shots),len(t),len(freqs),len(recAll)])

    # Rebuild coefficient data
    si = 0
    for shot in shots:
        for ff in range(len(freqs)):
            Htmp = daH.sel(shot=shot,frequency=freqs[ff])
            Ttmp = daT.sel(shot=shot,frequency=freqs[ff])
            Rdata[si,Ttmp,ff,rr] = Htmp
        si += 1
            
    rr += 1



## Put data into an Xarray, enforce peak spacing if desired

In [ ]:
rCoeffData = xr.DataArray(Rdata,
    dims = ("shot","time","frequency","receiver"),
    coords = {"shot": shots, "time":t, "frequency":freqs,"receiver":recAll},
    attrs = {"Description" : "Reflection inversion results",
            "Frequency Units" : "Hz",
            "Time Units" : "s"})

del Rdata 

## Method that takes whole window

In [ ]:

recLogs =  [   5,  10,  20,  30,  40,  50,  75, 100, 125, 150, 175, 205, 250, 300, 350, 400, 450, 500, 550, 600, 700, 850, 900,1000,1100,1199]
tStarts0 = [1855,1860,1865,1862,1867,1875,1895,1920,1950,1980,2020,2080,2170,2280,2410,2555,2700,2860,3020,3190,3546,4125,4315,4695,5085,5475]
tStarts1 = [2440,2440,2440,2440,2440,2440,2450,2455,2490,2525,2550,2575,2630,2710,2780,2875,2975,3070,3180,3310,3570,4010,4140,4420,4690,4950]
tAdjs0 =   [   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]
tAdjs1 =   [ 440, 440, 453, 430, 427, 420, 410, 405, 400, 370, 360, 350, 330, 310, 310, 270, 220, 210, 180, 120,  55,  20,   0,   0,   0,   0]

tWin = 50
shots = np.arange(0,1400,1)

freqs = [25] #[25, 32, 40, 50, 63, 80, 100, 125, 159, 200]
BWs   = [6,  8, 10, 12, 14, 18,  24,  30,  40,  40]

ci = complex(0,1)

fi = 0
freq = freqs[fi]

spread = 0

seafloorRC = np.zeros([len(recAll),len(shots),len(freqs)])
seafloorSEL = np.zeros([len(recAll),len(shots),len(freqs)])
crustRC = np.zeros([len(recAll),len(shots),len(freqs)])
crustSEL = np.zeros([len(recAll),len(shots),len(freqs)])
allSEL = np.zeros([len(recAll),len(shots),len(freqs)])
midSEL = np.zeros([len(recAll),len(shots),len(freqs)])

sigLen = 7501
dt = 1/Fs

for ff in freqs:
    print(ff)
    ri = 0

    for rr in recAll:
        print(rr)

        offset = rr*12.5+200
        srcBath = 2600 #srcBath[si] # 2600

        angle = 180/np.pi*np.arctan(2*srcBath/offset)
        rtmp = np.sqrt(srcBath**2 + offset**2)
        srcSig = SigExp(geometry, AirgunVols, angle, rtmp, ghost=False)
        srcSig = np.real(srcSig/np.max(srcSig))
        
        if len(srcSig < sigLen):
            srcSig = np.append(srcSig, np.zeros(sigLen-len(srcSig)))
        else:
            srcSig = srcSig[0:sigLen]
        
        f0 = freqs[fi]-BWs[fi]/2
        f1 = freqs[fi]+BWs[fi]/2
        f0_i = int(f0/Fs*len(srcSig))
        f1_i = int(f1/Fs*len(srcSig))

        SRCSIG = np.fft.fft(srcSig)
        SRCSIG[0:f0_i] = 0*SRCSIG[0:f0_i]
        SRCSIG[f1_i+1:-f1_i] = 0*SRCSIG[f1_i+1:-f1_i]
        SRCSIG[-f0_i+1:] = 0*SRCSIG[-f0_i+1:]
        srcSig = np.real(np.fft.ifft(SRCSIG))

        if rr in recLogs:
            rr_i = np.argmin(np.abs(recLogs-rr))
            tStart0 = tStarts0[rr_i]
            tStart1 = tStarts1[rr_i]
            tAdj0 = tAdjs0[rr_i]
            tAdj1 = tAdjs1[rr_i]
        else:
            recTmp = recLogs - rr
            neg_len = len(recTmp[recTmp<0])
            tStart0 = tStarts0[neg_len-1] + (tStarts0[neg_len] - tStarts0[neg_len-1]) * (rr-recLogs[neg_len-1]) / (recLogs[neg_len]-recLogs[neg_len-1])
            tStart1 = tStarts1[neg_len-1] + (tStarts1[neg_len] - tStarts1[neg_len-1]) * (rr-recLogs[neg_len-1]) / (recLogs[neg_len]-recLogs[neg_len-1])
            tAdj0 = tAdjs0[neg_len-1] + (tAdjs0[neg_len] - tAdjs0[neg_len-1]) * (rr-recLogs[neg_len-1]) / (recLogs[neg_len]-recLogs[neg_len-1])
            tAdj1 = tAdjs1[neg_len-1] + (tAdjs1[neg_len] - tAdjs1[neg_len-1]) * (rr-recLogs[neg_len-1]) / (recLogs[neg_len]-recLogs[neg_len-1])

        si = 0
        for shot in shots:
            
            winFloor = [ tStart0+tAdj0*shot/1400-tWin , tStart0+tAdj0*shot/1400+tWin ]
            winCrust = [ tStart1+tAdj1*shot/1400-tWin , tStart1+tAdj1*shot/1400+tWin ]
            winAll = [winFloor[0], winCrust[1]]

            # Find ref peaks from sea floor
            tmpInd0 = winFloor[0] + np.argmax( rCoeffData.isel( frequency=fi,receiver=ri,shot=shot,time=slice(int(winFloor[0]),int(winFloor[1]) ) ).to_numpy() )
            seafloorRC = rCoeffData.isel( frequency=fi,receiver=ri,shot=shot,time=slice(int(tmpInd0-tWin),int(tmpInd0+tWin)) )*1e5*rtmp**spread
            seafloorRC = np.append(seafloorRC,np.zeros(sigLen-len(seafloorRC)))
            seafloorSEL[ri,si,fi] = np.sqrt( np.sum( np.abs( np.real( np.fft.ifft( np.fft.fft(srcSig)*np.fft.fft(seafloorRC) ) ) )**2*dt ) )

            # Find ref peaks from crust
            tmpInd1 = winCrust[0] + np.argmax( rCoeffData.isel( frequency=fi,receiver=ri,shot=shot,time=slice(int(winCrust[0]),int(winCrust[1]) ) ).to_numpy() ) 
            crustRC = rCoeffData.isel( frequency=fi,receiver=ri,shot=shot,time=slice(int(tmpInd1-tWin),int(tmpInd1+tWin)) )*1e5*rtmp**spread
            crustRC = np.append(crustRC,np.zeros(sigLen-len(crustRC)))
            crustSEL[ri,si,fi] = np.sqrt( np.sum( np.abs( np.real( np.fft.ifft( np.fft.fft(srcSig)*np.fft.fft(crustRC) ) ) )**2 *dt) )

            # Find everything
            allRC = rCoeffData.isel( frequency=fi,receiver=ri,shot=shot )*1e5*rtmp**spread
            allRC = np.append(allRC,np.zeros(sigLen-len(allRC)))
            allSEL[ri,si,fi] = np.sqrt( np.sum( np.abs( np.real( np.fft.ifft( np.fft.fft(srcSig)*np.fft.fft(allRC) ) ) )**2*dt ) )
            
            # Find everything in between
            if np.abs(tmpInd1-tmpInd0) < tWin*2:
                midSEL[ri,si,fi] = 0
            else:
                if (tmpInd1-tmpInd0) > tWin*2:
                    midInd0 = int(tmpInd0+tWin)
                    midInd1 = int(tmpInd1-tWin)
                elif (tmpInd0-tmpInd1) > tWin*2:
                    midInd0 = int(tmpInd1+tWin)
                    midInd1 = int(tmpInd0-tWin)
                midRC = rCoeffData.isel( frequency=fi,receiver=ri,shot=shot,time=slice(midInd0,midInd1) )*1e5*rtmp**spread
                midRC = np.append(midRC,np.zeros(sigLen-len(midRC)))
                midSEL[ri,si,fi] = np.sqrt( np.sum( np.abs( np.real( np.fft.ifft( np.fft.fft(srcSig)*np.fft.fft(midRC) ) ) )**2*dt ) )

            si += 1
        ri += 1
    fi += 1

    

In [ ]:
offset = 200 + recAll*12.5
AllRefData = xr.DataArray(allSEL,
    dims = ("offset","shot","frequency bin"),
    coords = {"offset": offset, "shot":shots, "frequency bin":freqs},
    attrs = {"Description" : "Total acoustic energy (in Pa) from the full first reflection",
            "offset" : "m",
            "Frequency Bins Fc" : "Hz: 25, 32, 40, 50, 63, 80, 100, 125, 159, 200",
            "Frequency BWs" : "Hz: 6,  8, 10, 12, 14, 18,  24,  30,  40,  40"})

seafloorData = xr.DataArray(seafloorSEL,
    dims = ("offset","shot","frequency bin"),
    coords = {"offset": offset, "shot":shots, "frequency bin":freqs},
    attrs = {"Description" : "Total acoustic energy (in Pa) from the full first reflection",
            "offset" : "m",
            "Frequency Bins Fc" : "Hz: 25, 32, 40, 50, 63, 80, 100, 125, 159, 200",
            "Frequency BWs" : "Hz: 6,  8, 10, 12, 14, 18,  24,  30,  40,  40"})

midData = xr.DataArray(midSEL,
    dims = ("offset","shot","frequency bin"),
    coords = {"offset": offset, "shot":shots, "frequency bin":freqs},
    attrs = {"Description" : "Total acoustic energy (in Pa) from the full first reflection",
            "offset" : "m",
            "Frequency Bins Fc" : "Hz: 25, 32, 40, 50, 63, 80, 100, 125, 159, 200",
            "Frequency BWs" : "Hz: 6,  8, 10, 12, 14, 18,  24,  30,  40,  40"})

crustData = xr.DataArray(crustSEL,
    dims = ("offset","shot","frequency bin"),
    coords = {"offset": offset, "shot":shots, "frequency bin":freqs},
    attrs = {"Description" : "Total acoustic energy (in Pa) from the full first reflection",
            "offset" : "m",
            "Frequency Bins Fc" : "Hz: 25, 32, 40, 50, 63, 80, 100, 125, 159, 200",
            "Frequency BWs" : "Hz: 6,  8, 10, 12, 14, 18,  24,  30,  40,  40"})

recGood = xr.DataArray(recGood)

ds = xr.Dataset({"AllRefData": AllRefData, "seafloorData": seafloorData, "midData": midData, "crustData": crustData, "recGood":recGood},
    attrs = {"Description" : "Reflected acoustic energy from full reflection, seafloor, middle layers, and crust.",
       "Acoustic Units: Pa (need to square to get energy and convert to dB)"
       "Sampling Frequency" : "500 Hz",
       "Cruise" : "MGL2104",
       "Line" : "PD12",
       "Source,Receiver Depth" : "12 meters",
       "Freq. Bin Fc": "Hz: 25, 32, 40, 50, 63, 80, 100, 125, 159, 200",
       "Freq. Bin BWs": "Hz: 6,  8, 10, 12, 14, 18,  24,  30,  40,  40",
       "Inversion": "Modeled signal using experimental data, no ghost signal"})

ds.to_netcdf('/media/asd21/My Passport/AzureStuff/MGL2104_Layer_Results_WholeTrace2')